In [1]:
import pandas as pd

import plotly.express as px

from main import GameResult

In [2]:
def _sanity_check(x, CAT, animal_cols):
    cat_finished = x[CAT] == GameResult.FINISHED.name
    any_animal_finished = any(x[col] == GameResult.FINISHED.name for col in animal_cols)
    all_animals_chased = all(x[col] == GameResult.CHASED.name for col in animal_cols)
    assert cat_finished != any_animal_finished
    if any_animal_finished or all_animals_chased:
        assert any_animal_finished != all_animals_chased


def get_winner(x):
    CAT = "C"
    animal_cols = ["B", "S", "M"]
    _sanity_check(x, CAT, animal_cols)
    result = None
    for col in [CAT] + animal_cols:
        if x[col] == GameResult.FINISHED.name:
            if result is None:
                result = col
            else:
                raise ValueError("W?")
    return result

In [3]:
df = pd.read_csv("results.csv")
display(df.describe())
display(df.groupby(by="strategy").describe())

,epoch,position,game_result,shortcut_position,game_id,timestamp
count,463896.000000,463896.000000,463896.000000,347922.000000,463896.000000,4.638960e+05
mean,20.332799,15.835493,-0.758657,21.000000,1477.090167,1.746654e+12
std,13.752002,12.322123,5.376047,8.164978,854.946651,8.781858e+03
min,-1.000000,0.000000,-2.000000,11.000000,0.000000,1.746654e+12
25%,8.000000,6.000000,-2.000000,11.000000,747.000000,1.746654e+12
50%,19.000000,10.000000,-1.000000,21.000000,1459.000000,1.746654e+12
75%,31.000000,22.000000,-1.000000,31.000000,2205.000000,1.746654e+12
max,64.000000,50.000000,50.000000,31.000000,2999.000000,1.746654e+12


epoch                                                    \
                      count       mean        std  min  25%   50%   75%   max   
strategy                                                                        
CLOSEST_RUN_AWAY   161956.0  21.261553  14.133890 -1.0  9.0  20.0  33.0  63.0   
ONLY_ONE_RUN_AWAY  146008.0  19.207523  13.279535 -1.0  8.0  18.0  29.0  64.0   
RANDOM_SINGLE      155932.0  20.421825  13.708700 -1.0  8.0  20.0  32.0  64.0   

                   position             ... game_id         timestamp  \
                      count       mean  ...     75%     max     count   
strategy                                ...                             
CLOSEST_RUN_AWAY   161956.0  16.326589  ...  1742.0  1999.0  161956.0   
ONLY_ONE_RUN_AWAY  146008.0  15.255287  ...  2750.0  2999.0  146008.0   
RANDOM_SINGLE      155932.0  15.868706  ...   754.0   999.0  155932.0   

                                                                          \
                           mean          std           min           25%   
strategy                                                                   
CLOSEST_RUN_AWAY   1.746654e+12  3019.688764  1.746654e+12  1.746654e+12   
ONLY_ONE_RUN_AWAY  1.746654e+12  2763.358729  1.746654e+12  1.746654e+12   
RANDOM_SINGLE      1.746654e+12  3149.008097  1.746654e+12  1.746654e+12   

                                                             
                            50%           75%           max  
strategy                                                     
CLOSEST_RUN_AWAY   1.746654e+12  1.746654e+12  1.746654e+12  
ONLY_ONE_RUN_AWAY  1.746654e+12  1.746654e+12  1.746654e+12  
RANDOM_SINGLE      1.746654e+12  1.746654e+12  1.746654e+12  

[3 rows x 48 columns]

In [4]:
ds = []
for g_id in df.game_id.unique():
    df_game = df[df["game_id"] == g_id]
    last_epoch = df_game.epoch.max()
    df_last_epoch = df_game[df_game["epoch"] == last_epoch]
    d = {"end_epoch": last_epoch, "game_id": g_id, "strategy": df_last_epoch["strategy"].iloc[0]}
    for idx, row in df_last_epoch.iterrows():
        d[row["name"]] = GameResult(row["game_result"]).name
    ds.append(d)

df_game_result = pd.DataFrame(ds)
df_game_result["winner"] = df_game_result.apply(get_winner, axis=1)

In [5]:
df_game_result[df_game_result["C"] != GameResult.FINISHED.name]
px.histogram(df_game_result, "winner", color="strategy")


In [6]:
px.histogram(df_game_result, x="end_epoch", nbins=len(df_game_result.end_epoch.unique()), color="strategy")